# Импорт

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from Classes.Regression import Regression
from Classes.RegressionTrainer import RegressionTrainer
from Functions.lossFunctions import euclideError, euclideErrorGradient
from Classes.Polinomizer import Polinomizer
from Functions.gradientDescentAlgorithm import gradientDescentAlgorithm
from Functions.errorFunctions import MAE, MAPE, MSE, RMSE, R2
from Functions.regulatorFinctions import Lasso, Ridge, RidgeGrad
# from sklearn.preprocessing import scale


# Загрузка датасета для регрессии.

In [2]:
models = pd.read_csv('../../Data/energy_task_moded.csv')
model = models.drop('year', axis=1)
model = (model-model.mean())/model.std()
model


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,day,month,time
0,-0.367666,3.301180,-1.118616,1.843774,-0.520398,1.077616,-1.235032,1.686087,-0.908194,1.510576,...,0.958043,-0.154025,-2.983203,0.824355,1.207663,2.092365,0.366845,-0.584529,-1.567533,0.738423
1,-0.367666,3.301180,-1.118616,1.616766,-0.520398,1.060983,-1.235032,1.704523,-0.908194,1.608892,...,0.965270,-0.176708,-2.969656,0.824355,1.071676,1.767230,0.343008,-0.584529,-1.567533,0.762352
2,-0.465204,3.301180,-1.118616,1.517921,-0.520398,1.037367,-1.235032,1.748563,-0.944091,1.585227,...,0.950816,-0.197500,-2.956109,0.824355,0.935690,1.442094,0.319172,-0.584529,-1.567533,0.786280
3,-0.465204,4.561263,-1.118616,1.459284,-0.520398,1.028332,-1.235032,1.769047,-0.962039,1.546747,...,0.926726,-0.220182,-2.942562,0.824355,0.799703,1.116958,0.295335,-0.584529,-1.567533,0.810209
4,-0.367666,4.561263,-1.118616,1.526298,-0.520398,1.013546,-1.235032,1.769047,-0.962039,1.502111,...,0.926726,-0.242865,-2.929015,0.824355,0.663716,0.791822,0.271499,-0.584529,-1.567533,0.834138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,0.022483,-0.479068,2.415901,1.583259,2.530254,0.396436,2.458683,0.590195,1.881979,1.515963,...,1.261577,2.889209,-0.043505,-1.620295,-0.288190,-1.243811,2.273765,1.297945,1.422032,0.786280
19731,-0.075055,-0.479068,2.374391,1.568181,2.468238,0.409813,2.425451,0.608630,1.881979,1.515963,...,1.261577,2.870307,-0.043505,-1.597867,-0.220197,-1.173129,2.273765,1.297945,1.422032,0.810209
19732,1.680617,0.781015,2.374391,1.592473,2.411042,0.579492,2.383911,0.752018,1.881979,1.548286,...,1.261577,2.851405,-0.043505,-1.575439,-0.152204,-1.102447,2.273765,1.297945,1.422032,0.834138
19733,3.143676,0.781015,2.374391,1.691318,2.313197,0.645392,2.304155,0.629114,1.881979,1.562139,...,1.268202,2.813601,-0.043505,-1.553011,-0.084210,-1.031766,2.249928,1.297945,1.422032,0.858067


# Разбиваем на выборку

In [3]:
X_train, X_test, y_train, y_test = train_test_split(models.drop('Appliances', axis=1), models['Appliances'], test_size=0.5)
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    model.drop('Appliances', axis=1), model['Appliances'], test_size=0.5)


# Линейная регрессия

In [4]:
function_optimizer_error = lambda td, cd, w: gradientDescentAlgorithm(
    td=td, 
    cd=cd, 
    w=w, 
    lost_function=euclideError, 
    gradient=euclideErrorGradient, 
    nsteps=508, 
    merror=0.001, 
    step=0.00000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train, dtype=np.float64), np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)

my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                    }

print(my_line_regression_classic_errors)
print(y_predicted)


{'MAE': 155.00950879812913, 'RMSE': 186.07020336703602, 'MSE': 34622.12058105014, 'MAPE': 2.4167743423442336, 'R^2': 0.9996678261691294}
[  2.39808309 173.06195759  10.04528322 ...  84.52281543 346.78839952
 299.51367336]


In [5]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=euclideError,
    gradient=euclideErrorGradient,
    nsteps=5,
    merror=0.001,
    step=0.001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train1, dtype=np.float64),
              np.array(y_train1, dtype=np.float64))

y_predicted = regressor.predict(X_test1)

my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test1),
                                     'RMSE': RMSE(y_predicted, y_test1),
                                     'MSE': MSE(y_predicted, y_test1),
                                     'MAPE': MAPE(y_predicted, y_test1),
                                     'R^2': R2(y_predicted, np.array(y_test1))
                                     }

print(my_line_regression_classic_errors)
print(y_predicted)


{'MAE': 1.245310794916078, 'RMSE': 1.6447684268418195, 'MSE': 2.7052631779357137, 'MAPE': 6.241197538105737, 'R^2': 0.9997201371025213}
[ 0.03235366  2.65563317  0.86022054 ... -2.17500468 -0.41897414
  2.51481088]


In [6]:
alpha = 10
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w) + Lasso(w, alpha),
    gradient=lambda td, cd, w: euclideErrorGradient(
        td, cd, w) + alpha * w.shape[0],
    nsteps=1000,
    merror=0.01,
    step=0.00000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train, dtype=np.float64),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 1475.1775271613378,
 'RMSE': 1478.849399723369,
 'MSE': 2186995.5470621693,
 'MAPE': 24.16301196554196,
 'R^2': 0.979017383199738}

In [7]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w) + Ridge(w, 100),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w) + RidgeGrad(w, 100),
    nsteps=100,
    merror=0.01,
    step=0.000000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(np.array(X_train, dtype=np.float64),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 2318.0933269478937,
 'RMSE': 2329.787875224472,
 'MSE': 5427911.54354296,
 'MAPE': 37.0311282202149,
 'R^2': 0.9479231733704838}

In [8]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w),
    nsteps=3000,
    merror=0.0001,
    step=0.00000000000000000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(Polinomizer(np.array(X_train, dtype=np.float64), 3).polinomize(),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(Polinomizer(np.array(X_test, dtype=np.float64), 3).polinomize())
y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 179008316.59116694,
 'RMSE': 201443340.77727112,
 'MSE': 4.0579419543507784e+16,
 'MAPE': 3073744.604686639,
 'R^2': -389329740.1612344}

In [46]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w),
    nsteps=30000,
    merror=0.0001,
    step=0.00000000000000001)


regressor = Regression()
trainer = RegressionTrainer(regressor, function_optimizer_error)
trainer.train(Polinomizer(np.array(X_train1, dtype=np.float64), 3).polinomize(),
              np.array(y_train1, dtype=np.float64))

y_predicted = regressor.predict(Polinomizer(
    np.array(X_test1, dtype=np.float64), 3).polinomize())
y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test1),
                                     'RMSE': RMSE(y_predicted, y_test1),
                                     'MSE': MSE(y_predicted, y_test1),
                                     'MAPE': MAPE(y_predicted, y_test1),
                                     'R^2': R2(y_predicted, np.array(y_test1))
                                     }

my_line_regression_classic_errors


{'MAE': 34.781174232468395,
 'RMSE': 46.111033309136204,
 'MSE': 2126.227392836269,
 'MAPE': 209.94823802703044,
 'R^2': 0.7828000064710134}